In [77]:
#%pip install pysrt

In [78]:
#%pip install spacy

## Описание задачи

Даны списки субтитров и категории фильмов в зависимости от сложности: уровни A1,A2,B1,B2,C1.

На основании данных необходимо построить модель, которая будет определять уровень фильма на основании субтитров.

Данная задача относится к задачам мультиклассификации.

В связи с задачей мультиклассификации, метрика данной работы f1_score.

В работе будут использованы две модели - log.regression и catboost.

По завершению будет сохранен дамп модели.

Первоначально будет собран общий датасет на основании субтитров. После чего он будет очищен от дубликатов с фиксацией минимального уровня. Далее будет произведен EDA.

In [100]:
#импорт библиотек
import copy
import pandas as pd
import pysrt
import os
from collections import defaultdict
import re
import spacy
from spacy.tokenizer import Tokenizer 

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from catboost import CatBoostClassifier, Pool

from pickle import dump

## 1.Создание исходного датафрейма

In [80]:
#опрелеляем файлы в директории
path = 'C:/Users/angel/project_english_school/Subtitles_all_new/rar'
dir_list = os.listdir(path)
print('Файлы и директории в ', path, ':')
print(dir_list)

Файлы и директории в  C:/Users/angel/project_english_school/Subtitles_all_new/rar :
['01_Extra_English_-_Hectors_arrival.srt', '02_Extra_English_-_Hector_goes_shopping.Vie_Syned.srt', '03_Extra_English_-_Hector_has_a_date.Eng_Syned.srt', '04_Extra_English_-_Hector_looks_for_a_job.Vie_Syned.srt', '05_Extra_English_-_A_star_is_born.Eng_Syned.srt', '06_Extra_English_-_Bridget_wins_the_lottery.Vie_Syned.srt', '07_Extra_English_-_The_twin.Eng_Syned.srt', '08_Extra_English_-_The_landladys_cousin.Vie_Syned.srt', '09_Extra_English_-_Jobs_for_the_boys.Eng_Syned.srt', '10_Cloverfield_lane(2016).srt', '10_Extra_English_-_Annies_Protest.Vie_Syned.srt', '10_things_I_hate_about_you(1999).srt', '11_Extra_English_-_Holiday_time.Eng_Syned.srt', '12_Extra_English_-_Football_Crazy.Vie_Syned.srt', '13.Reasons.Why.S01E01.720p.WEBRiP.x265.ShAaNiG.srt', '13.Reasons.Why.S01E02.720p.WEBRiP.x265.ShAaNiG.srt', '13.Reasons.Why.S01E03.720p.WEBRiP.x265.ShAaNiG.srt', '13.Reasons.Why.S01E04.720p.WEBRiP.x265.ShAaNiG.s

In [81]:
#создаем датасет из субтитров и названия фильмов
results = defaultdict(list)
name = defaultdict(list)

for file in dir_list:
    sub = pysrt.open(path+'/'+file, encoding = 'latin-1')
    results['subtitles'].append(sub.text)
    name['Movie'].append(file.replace('.srt',''))
data = pd.DataFrame(results).join(pd.DataFrame(name))

In [82]:
#считаем данные по классам фильмов
movies_labels = pd.read_excel('C:/Users/angel/project_english_school/movies_labels_2.xlsx')

In [83]:
#создаем первый датасет

data1 = data.merge(movies_labels, 'inner', left_on = 'Movie', right_on = 'Movie').drop('id', axis = 1)

In [84]:
path = 'C:/Users/angel/project_english_school/Subtitles_all'
dir_list = os.listdir(path)
print('Файлы и директории в ', path, ':')
print(dir_list)


results = defaultdict(list)
name = defaultdict(list)
label = defaultdict(list)

for file in dir_list:
    if file in ['A2', 'B1', 'B2', 'C1']:
        dir_list_films = os.listdir(path+'/'+file)
        for film in dir_list_films:
            sub = pysrt.open(path+'/'+file + '/'+ film, encoding = 'latin-1')
            results['subtitles'].append(sub.text)
            name['Movie'].append(film.replace('.srt',''))
            label['Level'].append(file)
        
data2 = pd.DataFrame(results).join(pd.DataFrame(name)).join(pd.DataFrame(label))

Файлы и директории в  C:/Users/angel/project_english_school/Subtitles_all :
['A2', 'B1', 'B2', 'C1', 'Subtitles']


In [85]:
data = pd.concat([data1, data2]).reset_index().drop('index', axis=1)

In [86]:
data.head()

,subtitles,Movie,Level
0,"This is the story of Bridget and Annie,\nwho s...",01_Extra_English_-_Hectors_arrival,A1
1,This is the story of two girls\nwho share a fl...,02_Extra_English_-_Hector_goes_shopping.Vie_Syned,A1
2,This is the story of two girls\nwho share a fl...,03_Extra_English_-_Hector_has_a_date.Eng_Syned,A1
3,This is the story of two girls\nwho share a fl...,04_Extra_English_-_Hector_looks_for_a_job.Vie_...,A1
4,"This is the story of Bridget and Annie,\nwho s...",05_Extra_English_-_A_star_is_born.Eng_Syned,A1


## 2.EDA

In [87]:
#исследуем типы значений целевого признака
data['Level'].unique()

array(['A1', 'B1', 'A2', 'A2/A2+', 'B2', 'C1', 'B1, B2', 'A2/A2+, B1',
       'B2, C1'], dtype=object)

In [88]:
#уберем стоп слова, скорректируем данные субтитров, проведем лемматизацию, добавим дополнительные параметры для модели, скорректируем целевой признак

HTML = r'<.*?>' # html тэги меняем на пробел
TAG = r'{.*?}' # тэги меняем на пробел
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках меняем на пробел
UPPER = r'[[A-Za-z ]+[\:\]]' # указания на того кто говорит (BOBBY:)
LETTERS = r'[^a-zA-Z\'.,!? ]' # все что не буквы меняем на пробел 
SPACES = r'([ ])\1+' # повторяющиеся пробелы меняем на один пробел
DOTS = r'[\.]+' # многоточие меняем на точку
SYMB = r"[^\w\d'\s]" # знаки препинания кроме апострофа

nlp = spacy.load('en_core_web_sm',exclude=["tok2vec", "parser", "ner", "attrbute_ruler"])
stopwords = nlp.Defaults.stop_words
tokenizer = Tokenizer(nlp.vocab)

def clean_subs(subs):
    subs = subs[1:] # удаляем первый рекламный субтитр
    txt = re.sub(HTML, ' ', subs) # html тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) # комменты в скобках меняем на пробел
    txt = re.sub(UPPER, ' ', txt) # указания на того кто говорит (BOBBY:)
    txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
    txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
    txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
    txt = re.sub(SYMB, '', txt) # знаки препинания кроме апострофа на пустую строку
    txt = re.sub('www', '', txt) # кое-где остаётся www, то же меняем на пустую строку
    txt = txt.lstrip() # обрезка пробелов слева
    txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы   
    txt = txt.lower() # текст в нижний регистр
    return txt

def token_drop_stop(subs):
        
    new_tokens = tokenizer(subs)
    drop_stops = [w for w in new_tokens if w not in stopwords]  
    res_text = ''.join([token.lemma_ for token in nlp(str(drop_stops))])
    return res_text

def new_features(subs):
     new_tokens = tokenizer(subs)
     number = 0
     count = 0
     for w in new_tokens:
         number += len(w)
         count += 1
     return number/count

def level_update(row):
    global new_level
    if row == 'B1, B2':
        new_level = 'B1'
    elif row == 'A2/A2+':
        new_level = 'A2'
    elif row == 'A2/A2+, B1':
        new_level = 'A2'
    elif row == 'B2, C1':
        new_level = 'B2'
    else: new_level = row
    return new_level

def check_min_d(row):
    global x
    if row == 'A1':
        x = 1
    elif row == 'A2':
        x = 2
    elif row == 'B1':
        x = 3
    elif row == 'B2':
        x = 4
    else: x = 5

    return x



In [89]:
data['Level'] = data['Level'].apply(lambda row: level_update(row))
data['subtitles'] = data['subtitles'].apply(lambda row: clean_subs(row)).apply(lambda row: token_drop_stop(row))
#data['avg_word_length'] = data['subtitles'].apply(lambda row: new_features(row))

In [90]:
data['test'] = data['Level'].apply(lambda row: check_min_d(row))

In [91]:
data = copy.deepcopy(data.sort_values(by = ['test']).drop_duplicates(keep = 'first').drop('test', axis = 1).reset_index().drop('index', axis = 1))

In [92]:
data.head()

,subtitles,Movie,Level,avg_word_length
0,"[his,is,the,story,of,bridget,and,annie,who,sha...",01_Extra_English_-_Hectors_arrival,A1,2241.0
1,"[his,is,the,story,of,bridget,and,annie,who,sha...",16_Extra_English_-_Uncle_Nick.Vie_Syned,A1,5329.0
2,"[his,is,the,story,of,bridget,and,annie,who,sha...",17_Extra_English_-_Cyber_Stress.Eng_Syned,A1,5541.0
3,"[his,is,the,story,of,bridget,and,annie,who,sha...",18_Extra_English_-_Just_the_ticket.Vie_Syned,A1,2584.0
4,"[his,is,the,story,of,bridget,and,annie,who,sha...",19_Extra_English_-_Kung_fu_fighting.Eng_Syned,A1,5569.0


## 2.Разбиение на трейн и тест

In [93]:
#фиксируем переменные
RANDOM_STATE = 12345
TEST_SIZE = 0.30

In [94]:
#разбиваем датасет на трейн, тест, валид
X = data.drop(['Movie', 'Level'],axis=1)
Y = data['Level']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state = RANDOM_STATE, test_size = TEST_SIZE)


print('Относительная величина элементов в трейне',X_train.count()/X.count())
print('Относительная величина элементов в тесте',X_test.count()/X.count())

Относительная величина элементов в трейне subtitles          0.699288
avg_word_length    0.699288
dtype: float64
Относительная величина элементов в тесте subtitles          0.300712
avg_word_length    0.300712
dtype: float64


## 3.Создание пайплайна по логистической регресии

In [95]:
# создаем пайплайн логистической регресии и обучим модель
logistic_reg = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range = (2,10) ) ),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(n_jobs=4,C=3e5, solver='saga', 
                               multi_class='multinomial',
                               class_weight = 'balanced',
                               max_iter=500,
                               random_state=RANDOM_STATE)),
])
logistic_reg.fit(X_train['subtitles'], Y_train)
pred = logistic_reg.predict(X_test['subtitles'])

c:\Users\angel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [96]:
print(classification_report(Y_test, pred))
print(f"F1 Score: {f1_score(Y_test, pred, average='weighted')}")

              precision    recall  f1-score   support

          A1       1.00      0.83      0.91        12
          A2       0.74      0.91      0.82        54
          B1       0.55      0.46      0.50        24
          B2       0.75      0.80      0.78        61
          C1       1.00      0.44      0.62        18

    accuracy                           0.75       169
   macro avg       0.81      0.69      0.72       169
weighted avg       0.76      0.75      0.74       169

F1 Score: 0.7427837776950202


## 4.Обучение модели Catboost

In [97]:
def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(task_type = 'CPU', iterations = 5000,
                               eval_metric = 'TotalF1', od_type = 'Iter', random_state=RANDOM_STATE,
                               od_wait=500, **kwargs)
    return model.fit(train_pool, eval_set = test_pool,
                     verbose = 100, plot=True,
                     use_best_model = True)

In [98]:
train_pool = Pool(data=X_train, label=Y_train, 
                  text_features=['subtitles',])
valid_pool = Pool(data=X_test, label=Y_test, 
                  text_features=['subtitles',])

In [99]:
model = fit_model(train_pool, valid_pool, learning_rate=0.25,
                  dictionaries = [{
                      'dictionary_id':'Word',
                      'max_dictionary_size': '50000'
                  }],
                 feature_calcers = ['BoW:top_tokens_count=10000'])


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.2954271	test: 0.2749083	best: 0.2749083 (0)	total: 149ms	remaining: 12m 24s
100:	learn: 0.5798427	test: 0.3451250	best: 0.3490481 (47)	total: 637ms	remaining: 30.9s
200:	learn: 0.6545791	test: 0.3362460	best: 0.3500927 (172)	total: 1.41s	remaining: 33.8s
300:	learn: 0.6736347	test: 0.3262126	best: 0.3500927 (172)	total: 2.31s	remaining: 36s
400:	learn: 0.6986577	test: 0.3085549	best: 0.3500927 (172)	total: 3.15s	remaining: 36.1s
500:	learn: 0.7404118	test: 0.3075692	best: 0.3500927 (172)	total: 3.9s	remaining: 35s
600:	learn: 0.7650039	test: 0.2992522	best: 0.3500927 (172)	total: 4.56s	remaining: 33.4s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.3500927342
bestIteration = 172

Shrink model to first 173 iterations.


## Сохраним модель

In [101]:
#согласно метрике наилучшая модель - логистическая регрессия
#обучим модель на всем датасете
#затем сохраним модель для дальнейшего использования

logistic_reg.fit(X['subtitles'], Y)

with open('./main.pcl', 'wb') as model_file:
    dump(logistic_reg,model_file)

c:\Users\angel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
